<a href="https://colab.research.google.com/github/andres-chirinos/seprec-data/blob/main/notebook/analisis_seprec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import gzip
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import pandas as pd
import os

In [2]:
# PARAMS
zip_file = """/content/drive/Shareddrives/Repositorio de Datos/Data Lake/seprec-api/seprec_buscador_data.zip"""

In [3]:
drive.mount('/content/drive')

file = zip_file.split("/")[-1]
file_name = file.split(".")[0]

!unzip -q "{zip_file}" -d "/content"

Mounted at /content/drive


In [4]:
directory_path = "/content/data"
all_files = os.listdir(directory_path)
json_files = [os.path.join(directory_path, f) for f in all_files if f.endswith(".json")]

print(f"Found {len(json_files)} JSON files.")
print("First 5 JSON files:")
for file in json_files[:5]:
    print(file)

Found 102244 JSON files.
First 5 JSON files:
/content/data/result_a_3548.json
/content/data/result_e_3613.json
/content/data/result_a_6066.json
/content/data/result_i_5908.json
/content/data/result_u_16633.json


In [5]:
!pip install jsonpath-ng

In [6]:
import json
from jsonpath_ng import parse

def extract_data_from_json(file_path, queries):
    """
    Abre un archivo JSON, carga su contenido y concatena los resultados
    de cada query en registros fila por fila.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Primero obtenemos las listas por clave
    extracted = {}
    for name, query in queries.items():
        jsonpath_expression = parse(query)
        extracted[name] = [m.value for m in jsonpath_expression.find(data)]

    # Ahora alineamos por índice
    concatenated = []
    max_len = max(len(v) for v in extracted.values()) if extracted else 0

    for i in range(max_len):
        row = {}
        for key, values in extracted.items():
            row[key] = values[i] if i < len(values) else None
        concatenated.append(row)

    return concatenated


In [7]:
claves = {
    "id": "$.datos.filas[*].id",
    "id_establecimiento": "$.datos.filas[*].idEstablecimiento",
    "estado": "$.datos.filas[*].estado",
    "matricula": "$.datos.filas[*].matricula",
    "matricula_anterior": "$.datos.filas[*].matriculaAnterior",
    "id_estado_actualizacion": "$.datos.filas[*].codEstadoActualizacion.id",
    "codigo_estado_actualizacion": "$.datos.filas[*].codEstadoActualizacion.codigo",
    "nombre_estado_actualizacion": "$.datos.filas[*].codEstadoActualizacion.nombre",
    "id_tipo_unidad_economica": "$.datos.filas[*].codTipoUnidadEconomica.id",
    "codigo_tipo_unidad_economica": "$.datos.filas[*].codTipoUnidadEconomica.codigo",
    "nombre_tipo_unidad_economica": "$.datos.filas[*].codTipoUnidadEconomica.nombre",
    "id_direccion": "$.datos.filas[*].direccion.id",
    "id_direccion_departamento": "$.datos.filas[*].direccion.codDepartamento.id",
    "codigo_direccion_departamento": "$.datos.filas[*].direccion.codDepartamento.codigo",
    "nombre_direccion_departamento": "$.datos.filas[*].direccion.codDepartamento.nombre",
    "razon_social": "$.datos.filas[*].razonSocial"
}

extracted_data = extract_data_from_json("/content/data/result_a_3548.json", claves)
print(extracted_data[0])

{'id': '28690', 'id_establecimiento': '28690', 'estado': 'PENDIENTE', 'matricula': None, 'matricula_anterior': '00029556', 'id_estado_actualizacion': 17, 'codigo_estado_actualizacion': '1', 'nombre_estado_actualizacion': 'MATRICULA NO RENOVADA', 'id_tipo_unidad_economica': 18, 'codigo_tipo_unidad_economica': '01', 'nombre_tipo_unidad_economica': 'EMPRESA UNIPERSONAL', 'id_direccion': '28690', 'id_direccion_departamento': 420, 'codigo_direccion_departamento': '02', 'nombre_direccion_departamento': 'LA PAZ', 'razon_social': 'BARRACA VILLA ADELA'}


In [ ]:
from multiprocessing import Pool
from tqdm import tqdm

# función wrapper para map (para incluir queries)
def process_file(file_path):
    return extract_data_from_json(file_path, claves)

with Pool(processes=16) as pool:
    extracted_results = list(tqdm(pool.imap(process_file, json_files), total=len(json_files)))

print(f"Successfully extracted data from {len(extracted_results)} files.")


  4%|▍         | 4459/102244 [09:45<3:38:36,  7.45it/s]

In [ ]:
df = pd.DataFrame(extracted_results)
display(df.head())
df.info()